In [82]:
import pandas as pd
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 2000)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import seaborn as sns
sns.set(style="whitegrid")


In [83]:
df = pd.read_csv('/Users/jmirabito/Desktop/Learn.co/mod_2_final_project/citibike_modeling_df.csv')

In [84]:
df.set_index(['date','station_id'], inplace=True)

In [85]:
df.head()

rider_count  TAVG  PRCP  dist_PATH  dist_landmark  \
date       station_id                                                      
2019-08-01 3184                 86  79.0   0.0   0.010906       0.043861   
           3185                 91  79.0   0.0   0.010270       0.040780   
           3186                501  79.0   0.0   0.008522       0.038837   
           3187                121  79.0   0.0   0.003982       0.036882   
           3191                 22  79.0   0.0   0.025381       0.061002   

                       median_inc    mean_inc  population_zip  weekend  \
date       station_id                                                    
2019-08-01 3184        76967.0500  98752.3753         31104.0        0   
           3185        76967.0500  98752.3753         31104.0        0   
           3186        76967.0500  98752.3753         31104.0        0   
           3187        76967.0500  98752.3753         31104.0        0   
           3191        40861.4099  56704.7387         41745.0        0   

                       Monday  Tuesday  Wednesday  Thursday  Friday  Saturday  \
date       station_id                                                           
2019-08-01 3184             0        0          0         1       0         0   
           3185             0        0          0         1       0         0   
           3186             0        0          0         1       0         0   
           3187             0        0          0         1       0         0   
           3191             0        0          0         1       0         0   

                       Sunday  07087  07302  07304  07305  07306  07307  \
date       station_id                                                     
2019-08-01 3184             0      0      1      0      0      0      0   
           3185             0      0      1      0      0      0      0   
           3186             0      0      1      0      0      0      0   
           3187             0      0      1      0      0      0      0   
           3191             0      0      0      1      0      0      0   

                       07310  07311  fall  spring  summer  winter  
date       station_id                                              
2019-08-01 3184            0      0     0       0       1       0  
           3185            0      0     0       0       1       0  
           3186            0      0     0       0       1       0  
           3187            0      0     0       0       1       0  
           3191            0      0     0       0       1       0

In [86]:
# Removing features that are highly correlated
corr_matrix = df.corr().abs()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.shape

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
print('Columns dropped: %s'%(to_drop))

df.drop(columns=to_drop, inplace=True)

Columns dropped: ['mean_inc', 'population_zip']


In [87]:
# We must also drop one dummy variable from each category of dummies
df.drop(['Monday', '07087', 'fall'], axis=1, inplace=True)

## Creating an OLS Regression Model

In [88]:
# Run an OLS regression to determine which variables are most correlated with ridership
import statsmodels.api as sm

X = sm.add_constant(df.drop(['rider_count'], axis=1))
y = df.rider_count

model = sm.OLS(y, X)
results = model.fit()
print(results.summary()) 

                            OLS Regression Results                            
Dep. Variable:            rider_count   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.293
Method:                 Least Squares   F-statistic:                     416.4
Date:                Sun, 13 Sep 2020   Prob (F-statistic):               0.00
Time:                        21:10:01   Log-Likelihood:            -1.0287e+05
No. Observations:               20064   AIC:                         2.058e+05
Df Residuals:                   20043   BIC:                         2.060e+05
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -5.7919      4.044     -1.432

In [79]:
# Running the same regression using sklearn
from sklearn import linear_model as lm

reg = lm.LinearRegression().fit(X, y)
print(reg.score(X, y))
print(reg.coef_)
print(reg.intercept_)

# We ge the same R^2 value and coefficients using skleanr, but a different x intercept.

0.2929962779690214
[ 0.          0.25292969 -0.09800506 -0.21065323  0.11424796  0.21261664
 -0.02849831 -0.11765322 -0.06094452 -0.16324147 -0.0210783  -0.21899374
 -0.12275861 -0.06791957]
3.085285619355395e-17


In [80]:
# Standardize all columns in the df to facilitate interpretability
from scipy.stats import zscore

df = df.apply(zscore)

In [89]:
# Re-run the regression
X = sm.add_constant(df.drop(['rider_count'], axis=1))
y = df.rider_count

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            rider_count   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.293
Method:                 Least Squares   F-statistic:                     416.4
Date:                Sun, 13 Sep 2020   Prob (F-statistic):               0.00
Time:                        21:10:08   Log-Likelihood:            -1.0287e+05
No. Observations:               20064   AIC:                         2.058e+05
Df Residuals:                   20043   BIC:                         2.060e+05
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -5.7919      4.044     -1.432

In [90]:
# Our correlation remains unchanged, but our coefficients are now all in standard deviations, so we can more 
# easily interpret which variables are the strongest predictors of Citi Bike ridership in NJ. 

# We can also see that the R^2 value for our model is fairly weak, indicating that the variables in our model 
# are not very predictive of ridership as they're presented.


In [91]:
# Since some variables have p-values larger than 0.05, our alpha value, we will remove them from the model to see
# if model predictability increases. 

df.drop(['weekend', 'Tuesday', '07302'], axis=1, inplace=True)



In [92]:
# Re-run the regression
X = sm.add_constant(df.drop(['rider_count'], axis=1))
y = df.rider_count

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            rider_count   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.293
Method:                 Least Squares   F-statistic:                     438.3
Date:                Sun, 13 Sep 2020   Prob (F-statistic):               0.00
Time:                        21:10:09   Log-Likelihood:            -1.0287e+05
No. Observations:               20064   AIC:                         2.058e+05
Df Residuals:                   20044   BIC:                         2.059e+05
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -3.3349      5.644     -0.591

In [93]:
# Again, some variables have p-values larger than 0.05, our alpha value, so we will remove them from the model to see
# if model predictability increases. 

df.drop(['Saturday', '07310', '07311'], axis=1, inplace=True)



In [94]:
# Re-run the regression
X = sm.add_constant(df.drop(['rider_count'], axis=1))
y = df.rider_count

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            rider_count   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.293
Method:                 Least Squares   F-statistic:                     520.0
Date:                Sun, 13 Sep 2020   Prob (F-statistic):               0.00
Time:                        21:10:09   Log-Likelihood:            -1.0288e+05
No. Observations:               20064   AIC:                         2.058e+05
Df Residuals:                   20047   BIC:                         2.059e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           -11.1703      3.281     -3.405

In [95]:
# Again, some variables have p-values larger than 0.05, our alpha value, so we will remove them from the model to see
# if model predictability increases. 

df.drop(['Wednesday'], axis=1, inplace=True)



In [96]:
# Re-run the regression
X = sm.add_constant(df.drop(['rider_count'], axis=1))
y = df.rider_count

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            rider_count   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.293
Method:                 Least Squares   F-statistic:                     554.4
Date:                Sun, 13 Sep 2020   Prob (F-statistic):               0.00
Time:                        21:10:10   Log-Likelihood:            -1.0288e+05
No. Observations:               20064   AIC:                         2.058e+05
Df Residuals:                   20048   BIC:                         2.059e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           -10.6968      3.270     -3.271

In [97]:
# Again, some variables have p-values larger than 0.05, our alpha value, so we will remove them from the model to see
# if model predictability increases. 

df.drop(['Thursday', 'Friday'], axis=1, inplace=True)



In [98]:
# Re-run the regression
X = sm.add_constant(df.drop(['rider_count'], axis=1))
y = df.rider_count

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            rider_count   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.293
Method:                 Least Squares   F-statistic:                     639.2
Date:                Sun, 13 Sep 2020   Prob (F-statistic):               0.00
Time:                        21:10:10   Log-Likelihood:            -1.0288e+05
No. Observations:               20064   AIC:                         2.058e+05
Df Residuals:                   20050   BIC:                         2.059e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -9.8865      3.251     -3.041

In [99]:
# Using sklearn

reg = lm.LinearRegression().fit(X, y)
print(reg.score(X, y))
print(reg.coef_)
print(reg.intercept_)

# Again, using sklearn gives us the same coefficients and R^2 values.

0.2929962779690215
[ 0.00000000e+00  7.30243783e-01 -1.67931080e+01 -1.72651073e+03
  5.45566621e+02  4.05580143e-04 -3.95031615e+00 -1.65124327e+01
 -2.15566824e+01 -2.00622002e+01 -4.39662944e+00 -2.53850639e+01
 -1.28508945e+01 -7.83256537e+00]
-9.886491258496171


## Conclusion
In our final model, we see that average daily temperature, median income, distance to the PATH train, distance to a landmark, and spring seem to be the most influential variables for daily ridership. We further eplore the direct correlations between each of these features and ridership in our exploratory analyses and depict these relationships. Something to note here is that spring seems to have a very large negative effect on ridership, but this is likely due to the initial effect of COVID-19 on ridership between March 2020 and May 2020. We depict average daily ridership in our exploratory analysis as well. 

Overall, our OLS model has a fairly low R^2 value meaning that there are likely more predictive variables that we did not include in our model. In future analyses, we'd like to include neighborhood-level crime statistics, as well as number of restaurants and bars within a 100 foot radius of each station. 